<a href="https://colab.research.google.com/github/58191554/PointNet-Project/blob/main/pointNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install torch matplotlib numpy os

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement os (from versions: none)
ERROR: No matching distribution found for os


In [27]:
import sys
import os
from google.colab import drive
drive.mount('/content/drive')
root_folder = "/content/drive/MyDrive/PointNet/"

os.makedirs(root_folder, exist_ok=True)
os.chdir(root_folder)

import zipfile
import urllib.request


if not os.path.exists("pre_sample.zip"):
    print("Down Loading pre_sample.zip ...")
    urllib.request.urlretrieve("https://github.com/58191554/PointNet-Project/blob/main/pre_sample.zip", "pre_sample.zip")
    print("Finish DownLoading！")

if not os.path.exists("pre_sample"):
    print("unzipping pre_sample.zip...")
    with zipfile.ZipFile("pre_sample.zip", "r") as zip_ref:
        zip_ref.extractall(".")
    print("Finish unzipping")
else:
    print("pre_sample Exists")

def remove_ds_store(folder_path):
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file == ".DS_Store":
                file_path = os.path.join(root, file)
                os.remove(file_path)
                print("Deleted:", file_path)

remove_ds_store(root_folder+"pre_sample")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
pre_sample Exists


In [28]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import random

import plotly.graph_objects as go
import plotly.express as px

In [29]:
class_str_list = [
    "bathtub",
    "bed",
    "chair",
    "desk",
    "dresser",
    "monitor",
    "night_stand",
    "sofa",
    "table",
    "toilet"
]

class_to_idx = {}
idx_to_class = {}
for idx, class_str in enumerate(class_str_list):
  class_to_idx[class_str] = idx
  idx_to_class[idx] = class_str

for key, value in class_to_idx.items():
    print(f"key：{key}, value：{value}")

key：bathtub, value：0
key：bed, value：1
key：chair, value：2
key：desk, value：3
key：dresser, value：4
key：monitor, value：5
key：night_stand, value：6
key：sofa, value：7
key：table, value：8
key：toilet, value：9


### Visualization

In [55]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,mode='markers')]
    fig = visualize_rotate(data)    
    fig.update_traces(marker=dict(size=2,
            line=dict(width=2,
            color='DarkSlateGrey')),
            selector=dict(mode='markers'))
    fig.show()
    

### Dataset with sample function Class

In [31]:
# class PointDataSet(torch.utils.data.Dataset):
#   def __init__(self, root_dir, k=500):
#     self.root_dir = root_dir
#     self.k = k
#     self.items = []      #N
#     self.classes = []     #[0, 1, 2,...,10]
#     self.class_to_idx = class_to_idx
#     self.idx_to_class = idx_to_class
#     self.data = []       #NxMx3
#     self.label = []      #N
#     self.y = None       #Nx10

#     folders = os.listdir(root_dir)
        
#     # get different objects
#     for idx, item_folder in idx_to_class.items():

#       if idx > 2:break
#       print(idx, item_folder)
#       if not os.path.isdir(os.path.join(root_dir, item_folder)):
#         continue
            
#       # record class
#       self.classes.append(item_folder)
#       self.class_to_idx[item_folder] = idx
#       self.idx_to_class[idx] = item_folder
            
#       # get train data
#       train_folder = os.path.join(root_dir, item_folder, 'train')
#       if os.path.exists(train_folder):
#         train_files = [f for f in os.listdir(train_folder) if f.endswith('.off')]
#         self.items.extend([(os.path.join(train_folder, f), idx) for f in train_files])
#         self.label.extend([idx for f in train_files])

#       # get test data
#       test_folder = os.path.join(root_dir, item_folder, 'test')
#       if os.path.exists(test_folder):
#         test_files = [f for f in os.listdir(test_folder) if f.endswith('.off')]
#         self.items.extend([(os.path.join(test_folder, f), idx) for f in test_files])
#         self.label.extend([idx for f in test_files])

#       # break

#     # get tensor data
#     # self.load_to_RAM()

#   def __getitem__(self, index):
#     y = torch.zeros((1, len(self.classes)))
#     y[0, index]=1
#     points = self.read_off_file(self.items[index][0]), y

#   def __len__(self):
#       return len(self.items)

#   def load_to_RAM(self):
#     # reduce I/O, read into train/test_data list of torch tensor
#     print("Load classes: ", self.classes)
#     self.y = torch.zeros((len(self.items),len(self.classes)))
#     for i, (f, c) in tqdm(enumerate(self.items), dynamic_ncols=True, leave=True):
#       verts, faces = self.read_off_file(f)
#       self.y[i,c] = 1 
    
#   def read_off_file(self, file_path):
#     # return numpy
#     with open(file_path, 'r') as file:
#       if 'OFF' != file.readline().strip():
#         raise('Not a valid OFF header')
#       n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
#       verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
#       print("fucck")
#       faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
#       print("ass hole")
#       points = np.float32(self.sample(verts, faces, self.k))
      
#       # self.data.append(torch.from_numpy(points))

#     return points

#   def triangle_area(self, pt1, pt2, pt3):
#     # compute the area of the triangle by Heron's formula
#     side_a = np.linalg.norm(pt1 - pt2)
#     side_b = np.linalg.norm(pt2 - pt3)
#     side_c = np.linalg.norm(pt3 - pt1)
#     s = 0.5 * (side_a + side_b + side_c)
#     return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5 

#   def sample_point(self, pt1, pt2, pt3):
#     # barycentric coordinates on a triangle
#     s, t = sorted([random.random(), random.random()])
#     f = lambda i: s * pt1[i] + (t-s) * pt2[i] + (1-t) * pt3[i]
#     return [f(0), f(1), f(2)]

#   def sample(self, verts, faces, k):
#     # k: number of samples
#     # return a kx3 matrix of point clouds
#     areas = np.zeros((len(faces)))
#     verts = np.array(verts)
#     for i in range(len(areas)):
#       areas[i] = self.triangle_area(verts[faces[i][0]], verts[faces[i][1]], verts[faces[i][2]])
#       # sample k points from each face with weights by their areas
#       sampled_faces = (random.choices(faces, weights=areas, k=k))
#       pointcloud = np.zeros((k, 3))

#       for i in range(len(sampled_faces)):
#         pointcloud[i] = self.sample_point(verts[sampled_faces[i][0]],
#                           verts[sampled_faces[i][1]],
#                           verts[sampled_faces[i][2]])
#     print("shit")
#     return pointcloud


#   def get_min_points_num(self):
#     # return the minimum number of points among all the objects.
#     min_num = self[0][0].size(0)
#     for ts in self.data:
#       if(min_num > ts.size(0)):
#         min_num = ts.size(0)
#     return min_num

#   def get_unify_data(self):
#     min_num = self.get_min_points_num()
#     unify_data = []
#     unify_data.extend([random_sample(points, min_num) for points in self.data])
#     return unify_data

# def plot3d(points, plot_line = False):
#   fig = plt.figure()
#   ax = fig.add_subplot(111, projection='3d')

#   # get xyz coordinate data
#   x = points[:, 0]
#   y = points[:, 1]
#   z = points[:, 2]

#   # plot vertices
#   ax.scatter(x, y, z, c='b', marker='o', s=1)  # c: 点的颜色，marker: 点的形状，s: 点的大小

#   # plot line
#   if plot_line:
#     for i in range(points.shape[0]):
#       ax.plot([x[i], x[(i + 1) % points.shape[0]]],
#           [y[i], y[(i + 1) % points.shape[0]]],
#           [z[i], z[(i + 1) % points.shape[0]]], c='r', linewidth=0.5)

#   # set axis label
#   ax.set_xlabel('X')
#   ax.set_ylabel('Y')
#   ax.set_zlabel('Z')

#   plt.show()

# def random_sample(points, num):
#   if num > points.size(0):
#     return []
#   indices = torch.randperm(points.size(0))[:num]
#   # 从输入的 torch tensor 中根据索引选择对应的向量
#   selected_vectors = [points[i] for i in indices]

#   sample = torch.cat(selected_vectors, dim=0)
#   return sample

### Dataset Class 2

In [47]:
class PreSampleDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, valid=False, folder="train"):
        self.root_dir = root_dir
        self.valid = valid
        self.data = []
        self.y = []
        self.classNum = len(class_to_idx.keys())

        for item_folder in tqdm(class_to_idx.keys()):
            idx = class_to_idx[item_folder]
            new_dir = os.path.join(root_dir, item_folder, folder)
            for f in os.listdir(new_dir):
                if f.endswith('.pt'):
                    points = torch.load(os.path.join(new_dir,f))
                    self.data.append(points)
                    one_hot = torch.zeros((self.classNum))
                    one_hot[idx] = 1
                    self.y.append(one_hot)

    def __getitem__(self, index):
        return self.data[index], self.y[index]

    def __len__(self):
        return len(self.data)



In [43]:
data_path = os.path.join(root_folder, "pre_sample")
pre_sample_data_train = PreSampleDataset(data_path, folder="train")
pre_sample_data_test = PreSampleDataset(data_path, folder="test")

100%|██████████| 10/10 [00:01<00:00,  9.93it/s]


### Load Dataset

In [34]:
# data_path = "/content/drive/MyDrive/PointNet/ModelNet10"
# pointData = PointDataSet(data_path)
# points, label = pointData[2]
# plot3d(points)
# for i in range(len(pointData)):
#   print(pointData[i][1])

# train_ratio = 0.8
# test_ratio = 0.2
# batch_size = 16

# train_size = int(train_ratio * len(pointData))
# test_size = len(pointData) - train_size

# train_dataset, test_dataset = torch.utils.data.random_split(pointData, [train_size, test_size])

# # 创建训练数据和测试数据
# train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# uniData = UnifyDataset(pointData.get_unify_data(), pointData.label)
# print(len(uniData))
# train_unify_dataset, test_unify_dataset= torch.utils.data.random_split(uniData, [train_size, test_size])
# train_unify_dataloader = torch.utils.data.DataLoader(train_unify_dataset, batch_size=batch_size, shuffle=True)
# test_unify_dataloader = torch.utils.data.DataLoader(test_unify_dataset, batch_size=batch_size, shuffle=False)



### Load Dataset 2

In [57]:
pointCloud, label = pre_sample_data_train[534]
print(label)
x = pointCloud[0]
y = pointCloud[1]
z = pointCloud[2]
pcshow(*pointCloud.T)

tensor([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])


In [36]:
class MLP(nn.Module):
  def __init__(self, layers):
    super(MLP, self).__init__()
    self.layers = nn.ModuleList()  # 使用 ModuleList 存储每一层的 Module
    for i in range(1, len(layers)):
      self.layers.append(nn.Linear(layers[i-1], layers[i]))
    
  def forward(self, x):
    for layer in self.layers:
      x = torch.relu(layer(x))
      x = torch.softmax(x, dim=0)
    return x

Try MLP

Train MLP 